In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
train_data = pd.read_csv("https://raw.githubusercontent.com/HsinhanTsai/pycs4001/main/project2_%E8%94%A1%E6%98%95%E7%BF%B0/train.csv")
train_data = train_data[train_data["Cabin"].notna()]
train_data["deck"] = np.array(map(lambda x:x[0],train_data["Cabin"]))
train_data["side"] = np.array(map(lambda x:x[-1],train_data["Cabin"]))
train_data["num"] = np.array(map(lambda x:int(x[2:-2]),train_data["Cabin"]))
train_data = train_data.drop(["Cabin","Name","PassengerId"],axis=1)
for col in ["HomePlanet","CryoSleep","VIP","Destination"]:
  train_data[col] = train_data[col].fillna("unknown")
for col in ["Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]:
  train_data[col] = train_data[col].fillna(train_data[col].mean())
train_data.shape

(8494, 14)

In [ ]:
test_data = pd.read_csv("https://raw.githubusercontent.com/HsinhanTsai/pycs4001/main/project2_%E8%94%A1%E6%98%95%E7%BF%B0/test.csv")

def f1(x):
  if type(x) == type("hi"):
      return x[0]
  else:
      return "F"

def f2(x):
  if type(x) == type("hi"):
      return x[-1]
  else:
      return "S"

def f3(x):
  if type(x) == type("hi"):
      return int(x[2:-2])
  else:
      return 610

test_data["deck"] = np.array(map(f1,test_data["Cabin"]))
test_data["side"] = np.array(map(f2,test_data["Cabin"]))
test_data["num"] = np.array(map(f3,test_data["Cabin"]))
for col in ["HomePlanet","CryoSleep","VIP","Destination"]:
  test_data[col] = test_data[col].fillna("unknown")
for col in ["Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]:
  test_data[col] = test_data[col].fillna(test_data[col].mean())
test_ids = test_data["PassengerId"]
test_data = test_data.drop(["Cabin","Name","PassengerId"],axis=1)
test_data.shape

(4277, 13)

In [ ]:
X1 = train_data[["HomePlanet","CryoSleep","Destination","VIP","deck","side"]]
X2 = train_data[["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck","num"]]
Y = train_data["Transported"]
print(X1.shape)
print(X2.shape)
print(Y.shape)
print(test_data.shape)

(8494, 6)
(8494, 6)
(8494,)
(4277, 13)


In [ ]:
dummy_x = pd.get_dummies(X1)
dummy_x = dummy_x.merge(X2, how='inner', left_index=True, right_index=True)
print(dummy_x.shape)
dummy_x

(8494, 30)


,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_unknown,CryoSleep_False,CryoSleep_True,CryoSleep_unknown,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,...,deck_G,deck_T,side_P,side_S,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,num
0,0,1,0,0,1,0,0,0,0,1,...,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0
1,1,0,0,0,1,0,0,0,0,1,...,0,0,0,1,109.0,9.0,25.0,549.0,44.0,0
2,0,1,0,0,1,0,0,0,0,1,...,0,0,0,1,43.0,3576.0,0.0,6715.0,49.0,0
3,0,1,0,0,1,0,0,0,0,1,...,0,0,0,1,0.0,1283.0,371.0,3329.0,193.0,0
4,1,0,0,0,1,0,0,0,0,1,...,0,0,0,1,303.0,70.0,151.0,565.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,1,0,0,1,0,0,1,0,0,...,0,0,1,0,0.0,6819.0,0.0,1643.0,74.0,98
8689,1,0,0,0,0,1,0,0,1,0,...,1,0,0,1,0.0,0.0,0.0,0.0,0.0,1499
8690,1,0,0,0,1,0,0,0,0,1,...,1,0,0,1,0.0,0.0,1872.0,1.0,0.0,1500
8691,0,1,0,0,1,0,0,1,0,0,...,0,0,0,1,0.0,1049.0,0.0,353.0,3235.0,608


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
sc_x = scaler.fit_transform(dummy_x)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
depth = []
for i in tqdm(range(2,50,1)):
  rf = RandomForestClassifier(max_depth=i)
  scores = cross_val_score(rf, sc_x, Y, cv=5)
  depth.append(scores.mean())

100%|██████████| 48/48 [03:24<00:00,  4.25s/it]


In [ ]:
final_depth = np.argmax(np.array(depth)) + 2
print(final_depth)

11


In [ ]:
estimators = []
for i in tqdm(range(50,200,1)):
  rf = RandomForestClassifier(n_estimators=i, max_depth=final_depth)
  scores = cross_val_score(rf, sc_x, Y, cv=5)
  estimators.append(scores.mean())

100%|██████████| 150/150 [10:50<00:00,  4.33s/it]


In [ ]:
final_estimators = np.argmax(np.array(estimators)) + 50
print(final_estimators)

168


In [ ]:
final_rf = RandomForestClassifier(n_estimators=final_estimators, max_depth=final_depth)
final_rf.fit(sc_x, Y)

RandomForestClassifier(max_depth=11, n_estimators=168)

In [ ]:
test_data1 = test_data[["HomePlanet","CryoSleep","Destination","VIP","deck","side"]]
test_data2 = test_data[["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck","num"]]
dummy_test_data = pd.get_dummies(test_data1)
dummy_test_data = dummy_test_data.merge(test_data2, how='inner', left_index=True, right_index=True)
sc_test_data_x = scaler.transform(dummy_test_data)
submission = final_rf.predict(sc_test_data_x)

In [ ]:
print(submission)

[ True False  True ...  True  True  True]


In [ ]:
submission_data = pd.DataFrame({"PassengerId": test_ids.values, "Transported": submission })
submission_data.to_csv("submission.csv", index=False)